In [1]:
import sys

!'{sys.executable}' -m pip install --upgrade pip
!'{sys.executable}' -m pip install gurobipy
!'{sys.executable}' -m pip install cvxpy
!'{sys.executable}' -m pip install numpy
!'{sys.executable}' -m pip install jupyter-matlab-proxy
!'{sys.executable}' -m pip install matplotlib
!'{sys.executable}' -m pip install multiprocess
!'{sys.executable}' -m pip install matlabengine

The filename, directory name, or volume label syntax is incorrect.
The filename, directory name, or volume label syntax is incorrect.
The filename, directory name, or volume label syntax is incorrect.
The filename, directory name, or volume label syntax is incorrect.
The filename, directory name, or volume label syntax is incorrect.
The filename, directory name, or volume label syntax is incorrect.
The filename, directory name, or volume label syntax is incorrect.
The filename, directory name, or volume label syntax is incorrect.


Generic probability distribution histogram code + some helper functions here

In [1]:
import math
import numpy as np
#import multiprocessing as mp
import multiprocess as mp
import itertools
from collections import defaultdict

def push_multinomial(n, k, occ=1):
    return math.lgamma(n + k * occ + 1) - math.lgamma(n + 1) - occ * math.lgamma(k + 1)

class DistHistogram:
    def __init__(self, hist = defaultdict(int)):
        self.hist = hist

    # If the histogram actually represents a prob distribution
    def check_integrity(self):
        if not math.isclose(1.0, sum(self.hist[p] * p for p in self.hist)):
            raise Exception("Bad histogram: {}".format(str(self.hist)))

    # Generate a fingerprint
    def generate_fingerprint(self, k):
        self.check_integrity()
        n = sum(self.hist[p] for p in self.hist)
        dist = np.concatenate([[p] * self.hist[p] for p in self.hist])
        hist = np.random.multinomial(k, dist, size=1)
        unique, counts = np.unique(hist, return_counts=True)
        fingerprint = dict(zip(unique, counts))
        if (zero := n - sum(fingerprint[f] for f in fingerprint)) != 0:
            fingerprint[0] = zero
        return fingerprint

    def __str__(self):
        return str(hist)

# Get true probability of obtaining a fingerprint 
# Speedup calculation with vectorization
def get_probabilities(dists, fingerprint):
    # Needs to have all distributions of the same ??-point structure
    pts = {len(dist.hist) for dist in dists}
    if len(pts) > 1:
        raise Exception("All distributions need to have the same ??-point structure")
    k, pts = sum(fingerprint[f] * f for f in fingerprint), pts.pop()
    log_zero_arr = lambda : np.full((1, m), -np.inf, dtype=float)

    m = len(dists)
    lps, ys = np.log(np.transpose([list(dist.hist.keys()) for dist in dists])), np.array([list(dist.hist.values()) for dist in dists])
    coefficients = np.full((1, m), math.lgamma(k + 1))
    for f in fingerprint:
        coefficients -= fingerprint[f] * math.lgamma(f + 1)
    for y in np.transpose(ys):
        coefficients += np.vectorize(math.lgamma)(y + 1)
    dp = defaultdict(log_zero_arr, {np.zeros(pts).tobytes(): coefficients})

    # Partition lazy generator
    def partition(size, array, value, occ):
        ind = array.size
        if ind == pts - 1:
            i = size
            new_value = value - math.lgamma(i + 1) + occ * i * lps[ind]
            new_array = np.concatenate([array, [i]])
            yield (new_array, new_value)
        else:
            for i in range(size + 1):
                new_value = value - math.lgamma(i + 1) + occ * i * lps[ind]
                new_array = np.concatenate([array, [i]])
                yield from partition(size - i, new_array, new_value, occ)

    for occ, frq in fingerprint.items():
        new_dp = defaultdict(log_zero_arr)
        for arr, val in partition(frq, np.array([]), np.full((1, m), 0.0, dtype=float), occ):
            for prev_arr_byte, prev_val in dp.items():
                prev_arr = np.frombuffer(prev_arr_byte)
                idx = (arr + prev_arr).tobytes()
                new_dp[idx] = np.logaddexp(new_dp[idx], val + prev_val)
        dp = new_dp
    ans = np.exp([dp[np.array(ys[i], dtype=float).tobytes()][0, i] for i in range(m)])
    return ans

def get_probabilities_no_multithread(dists, fingerprints):
    return np.transpose([get_probabilities(dists, f) for f in fingerprints])

def get_probabilities_multithread(dists, fingerprints):
    pool = mp.Pool(processes=16)
    return np.transpose(pool.starmap(get_probabilities, zip(itertools.repeat(dists), fingerprints)))

def get_probabilities_nonuniform(dists, fingerprints):
    #print("dists: ", [(a.hist, b.hist) for (a,b) in dists])
    #print("fingerprints: ", fingerprints)

    d1s = [a for (a,_) in dists]
    d2s = [b for (_,b) in dists]

    #print("d1s: ", str([d.hist for d in d1s]))
    #print("d2s: ", str([d.hist for d in d2s]))

    p1s = [get_probabilities(d1s, f1) for (f1, _) in fingerprints]
    p2s = [get_probabilities(d2s, f2) for (_, f2) in fingerprints]
    """
    print("p1s: ", p1s)
    print("p2s: ", p2s)

    print("--------- Begin Test ----------")
    for (f1, f2) in fingerprints:
        print("Computing get_probabilities(" + str([d.hist for d in d2s]) + ", " + str(f2) + "):")
        print("    Result: " + str(get_probabilities(d2s, f2)))
    print("---------- End Test -----------")
    """

    def jprob(k,j):
        return np.exp(math.lgamma(k + 1) - math.lgamma(k - j + 1) - math.lgamma(j + 1) - k * math.log(2))

    ans = []
    for i in range(len(fingerprints)):
        (f1, f2) = fingerprints[i]
        j=sum(f1[f] * f for f in f1)
        k=j + sum(f2[f] * f for f in f2)

        ans.append([jprob(k,j)*a*b for a,b in zip(p1s[i],p2s[i])])
    return np.transpose(ans)

Generic tester + semilinear tester

In [3]:
class SemilinearTester: #this might not be entirely clean if the worst case distribution changes due to probabilism
    def __init__(self, c):
        self.c = c

    def get_coefficient(self, f):
        return sum([cnt * self.c[0][i] for i, cnt in f[0].items()])+sum([cnt * self.c[1][i] for i, cnt in f[1].items()])

    def optimize(self, lo, hi, fs, prob_hypo, prob_alts):
        def binary_search(lo, hi, pred, eps=1e-9):
            # assuming that pred(lo) is true and pred(hi) is false
            while lo + eps < hi:
                mi = (lo + hi) / 2
                if pred(mi):
                    lo = mi
                else:
                    hi = mi
            return lo, hi

        def predicate(t):
            verdict = np.array([self.get_coefficient(f) >= t for f in fs], dtype=float)
            t1 = (prob_hypo @ verdict)[0]
            t2 = np.max(prob_alts @ (1 - verdict))
            return t1 > t2
    
        self.lo, self.hi = binary_search(lo, hi, predicate)
        verdict_lo = np.array([self.get_coefficient(f) >= self.lo for f in fs], dtype=float)
        t1_lo = (prob_hypo @ verdict_lo)[0]
        t2_lo = np.max(prob_alts @ (1 - verdict_lo))
        verdict_hi = np.array([self.get_coefficient(f) >= self.hi for f in fs], dtype=float)
        t1_hi = (prob_hypo @ verdict_hi)[0]
        t2_hi = np.max(prob_alts @ (1 - verdict_hi))
        # solve t1_lo + (t1_hi - t1_lo) * self.tb = t2_lo + (t2_hi - t2_lo) * self.tb
        self.tb = (t1_lo - t2_lo) / ((t1_lo - t1_hi) - (t2_lo - t2_hi))
        return np.log(t1_lo + (t1_hi - t1_lo) * self.tb)

    def single_verdict(self, f):
        if (coef := self.get_coefficient(f)) < self.lo:
            return 0
        elif coef > self.hi:
            return 1
        else:
            return self.tb # this can be changed to a bernoulli sample with probability self.tb

    def multiple_verdicts(self, fs):
        return np.array([self.single_verdict(f) for f in fs], dtype=float)


Generating all alternative two-point distributions, sampling fingerprints, and generating probabilities

In [4]:
import copy

n, k = 10,10
eps = 0.9
fg_size = 20000

hypo = DistHistogram({1/n: n})
alts = []
for q in range(1, n):
    # x1 * q + x2 * (n - q) = 1
    # (1 / n - x1) * q + (x2 - 1 / n) * (n - q) = eps
    x1, x2 = (2 * q - n * eps) / (2 * n * q), (2 * n + n * eps - 2 * q) / (2 * n * (n - q))
    if 0 < x1 <= 1 and 0 < x2 <= 1:
        alts.append(DistHistogram({x1: q, x2: n - q}))

def generate_fingerprint(n, k, mi, dct):
    if n == 1:
        dct[k] += 1
        yield dict(dct)
    else:
        if n * (mi + 1) <= k:
            yield from generate_fingerprint(n, k, mi + 1, copy.deepcopy(dct))
        dct[mi] += 1
        yield from generate_fingerprint(n - 1, k - mi, mi, dct)

fingerprints = list(generate_fingerprint(n, k, 0, defaultdict(int)))

def generate_fingerprint_nonuniform(n,k):
    ans=[]
    for j in range(0,k+1):
        ans += [(f1,f2) for f1 in generate_fingerprint(2, j, 0, defaultdict(int)) for f2 in generate_fingerprint(n,k-j,0,defaultdict(int))]

    return ans

nonuniform_hypo = (DistHistogram({1/2: 2}), DistHistogram({1/n: n}))
nonuniform_alts = (DistHistogram({1/2-eps/2: 1, 1/2+eps/2:1}), DistHistogram({1/n: n}))

fingerprints_nonuniform = generate_fingerprint_nonuniform(n,k)

# seen = set()
# fingerprints = []
# while len(fingerprints) < fg_size:
#     f = hypo.generate_fingerprint(k)
#     t = tuple(f.items())
#     if t not in seen:
#         seen.add(t)
#         fingerprints.append(f)

# fingerprints = [hypo.generate_fingerprint(k) for _ in range(fg_size)]
print(len(fingerprints_nonuniform))
#print(fingerprints_nonuniform)
prob_hypo = get_probabilities_nonuniform([nonuniform_hypo], fingerprints_nonuniform)
print(prob_hypo)
prob_hypo = prob_hypo / prob_hypo.sum(axis=1)[:, np.newaxis] # normalization step
# prob_alts = np.array([[alt.get_probability(fingerprint) for fingerprint in fingerprints] for alt in alts])
prob_alts = get_probabilities_nonuniform([nonuniform_alts], fingerprints_nonuniform)
prob_alts = prob_alts / prob_alts.sum(axis=1)[:, np.newaxis] # normalization step

253
[[3.54375000e-07 1.59468750e-05 1.11628125e-04 2.12625000e-05
  1.86046875e-04 1.48837500e-04 1.24031250e-05 6.97675781e-05
  1.86046875e-04 3.10078125e-05 4.65117188e-05 3.72093750e-06
  2.79070313e-06 3.72093750e-05 3.72093750e-05 2.79070313e-05
  1.24031250e-05 7.44187500e-06 6.20156250e-07 3.10078125e-06
  1.55039062e-06 1.37812500e-06 6.20156250e-06 1.86046875e-06
  7.75195313e-07 1.24031250e-06 6.20156250e-07 5.90625000e-08
  1.47656250e-07 1.10742188e-07 1.77187500e-07 4.42968750e-08
  8.85937500e-08 5.90625000e-08 2.53125000e-08 3.16406250e-09
  1.10742188e-09 1.84570312e-09 1.05468750e-09 3.95507813e-10
  8.78906250e-11 9.76562500e-13 3.54375000e-05 6.37875000e-04
  2.23256250e-03 4.96125000e-04 1.86046875e-03 1.86046875e-03
  1.86046875e-04 2.79070313e-04 1.11628125e-03 2.48062500e-04
  3.72093750e-04 3.72093750e-05 6.20156250e-05 1.24031250e-04
  9.30234375e-05 6.20156250e-05 3.72093750e-05 4.13437500e-06
  1.96875000e-06 8.85937500e-06 2.65781250e-06 2.21484375e-06
  3.

Solving linear program

In [6]:
import matlab.engine
import io

# --2.783246248005155
# -2.5430245416741255 deterministic tester

lo, hi = -1e6, 1e6
p = 2/(n+2)
q = 0.5

eng = matlab.engine.start_matlab()
domain_fractions = matlab.double([p,1-p])
probability_fractions = matlab.double([q,1-q])
c_matlab = np.transpose(eng.get_coefs2(n * 1.0, k * 1.0, eps * 1.0,domain_fractions,probability_fractions, stdout=io.StringIO()))
#if len(c_matlab) < k + 1:
#    c_matlab = np.pad(c_matlab, (0, k + 1 - len(c_matlab)), 'constant')
#print(c_matlab)
matlab_tester = SemilinearTester(c_matlab)
print("Our tester:\n")
print(matlab_tester.optimize(lo, hi, fingerprints_nonuniform, prob_hypo, prob_alts))
print(matlab_tester.lo, matlab_tester.hi, matlab_tester.tb)

c_chi = np.array([[((i - (k*q) / (n*p)) ** 2 / ((k*q) / (n*p))) for i in range(k + 1)],[((i - (k*(1-q)) / (n*(1-p))) ** 2 / ((k*(1-q)) / (n*(1-p)))) for i in range(k + 1)]])
chi_squared_tester = SemilinearTester(c_chi)
print("chi_square:\n")
print(chi_squared_tester.optimize(lo, hi, fingerprints_nonuniform, prob_hypo, prob_alts))
print(chi_squared_tester.lo, chi_squared_tester.hi, chi_squared_tester.tb)

#c_tv = np.array([abs(i - k / n) for i in range(k + 1)])
#tv_tester = SemilinearTester(c_tv)
#print(tv_tester.optimize(lo, hi, fingerprints, prob_hypo, prob_alts))
#print(tv_tester.lo, tv_tester.hi, tv_tester.tb)

#c_singleton = -np.array([i == 1 for i in range(k + 1)], dtype=float)
#singleton_tester = SemilinearTester(c_singleton)
#print(singleton_tester.optimize(lo, hi, fingerprints, prob_hypo, prob_alts))
#print(singleton_tester.lo, singleton_tester.hi, singleton_tester.tb)

Our tester:

-1.1533220635953263
-0.6062670028228467 -0.6062670019346683 0.1582159929645564
chi_square:

-1.1795495677261283
10.666666666381275 10.666666667269453 0.25783328620558826


In [ ]:
print(*list(zip(x.value, fingerprints)), sep='\n')